#  Финальная работа по курсу SQL

В качестве образца принята схема базы данных предназначенная для системы тестирования.


In [ ]:
Для работы используется Docker контейнер с установленной БД postgres 
(Один из тех, что использовался в процессе обучения и выполнения ДЗ)

In [1]:
import os
import logging
import psycopg2
import psycopg2.extensions
from pymongo import MongoClient
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, Float, MetaData, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

/home/user/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)


In [35]:
logger.info("Создаём подключёние к Postgres")
params = {
    "host": '127.0.0.1',
    "port": '5433',
    "user": 'postgres'
}

2018-08-26 18:02:47,224 : INFO : Создаём подключёние к Postgres


In [36]:
conn = psycopg2.connect(**params)


In [37]:
psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()

In [38]:
## Создание таблиц
sql1 = "CREATE TABLE IF NOT EXISTS person (\
    id  INTEGER PRIMARY KEY, \
    first_name VARCHAR(45),\
    last_name VARCHAR(45),\
    second_name VARCHAR(255),\
    e_mail VARCHAR(255) NOT NULL,\
    login VARCHAR(45) NOT NULL ,\
    password VARCHAR(45) NOT NULL);"

sql2 = "CREATE TABLE IF NOT EXISTS event_type (\
    id  INTEGER PRIMARY KEY, \
    type VARCHAR(45),\
    description TEXT );\
CREATE TABLE IF NOT EXISTS event (\
    id  INTEGER PRIMARY KEY, \
    description TEXT,\
    start_date timestamp,\
    finish_date timestamp,\
    id_creater integer REFERENCES person(id),\
    id_type_event integer REFERENCES event_type(id));\
" 
sql3= """CREATE TABLE IF NOT EXISTS type_stratum (
    id  INTEGER PRIMARY KEY, 
    type VARCHAR(45),
    description TEXT );
"""

sql4= """CREATE TABLE IF NOT EXISTS stratum (
    id  INTEGER PRIMARY KEY, 
    name VARCHAR(45) NOT NULL,
    description TEXT,
    id_creater integer REFERENCES person(id),
    ParentId integer REFERENCES stratum(id),
    id_type_stratum integer REFERENCES type_stratum(id));

"""
sql5= """CREATE TABLE IF NOT EXISTS test (
    id  INTEGER PRIMARY KEY, 
    short_name  VARCHAR(45) NOT NULL,
    description TEXT,
    pass_level_norm integer NOT NULL,
    pass_level_good integer NOT NULL, 
    pass_level_perf integer NOT NULL,
    id_creater integer REFERENCES person(id),
    ParentId integer REFERENCES test (id));
"""
sql6= """CREATE TABLE IF NOT EXISTS type_question(
    id  INTEGER PRIMARY KEY, 
    type VARCHAR(45),
    description TEXT );

CREATE TABLE IF NOT EXISTS question (
    id  INTEGER PRIMARY KEY, 
    text TEXT NOT NULL,
    id_creater integer REFERENCES person(id),
    id_type_question  integer REFERENCES type_question(id));
"""
sql7= """CREATE TABLE IF NOT EXISTS answer (
id  INTEGER PRIMARY KEY, 
    text TEXT NOT NULL,
    explanation TEXT,
    correct integer,
    id_creater integer REFERENCES person(id),
    id_question  integer REFERENCES question(id));
"""
sql8 = """CREATE TABLE IF NOT EXISTS resault_person (
    value  VARCHAR(45),
    data  timestamp,
    id_event integer REFERENCES event(id),
    id_answer  integer REFERENCES answer(id),
    id_person integer REFERENCES person(id));
"""

sql9 = """CREATE TABLE IF NOT EXISTS event_test (
    id_event  integer REFERENCES event(id),
    id_test integer REFERENCES test(id));

 CREATE TABLE IF NOT EXISTS event_person (
    id_event  integer REFERENCES event(id),
    id_person integer REFERENCES person(id),
    connect_time timestamp);

CREATE TABLE IF NOT EXISTS event_stratum (
    id_event  integer REFERENCES event(id),
    id_stratum integer REFERENCES stratum(id),
    connect_time timestamp);

CREATE TABLE IF NOT EXISTS test_question (
   id_test integer REFERENCES test(id),
   id_question  integer REFERENCES question(id));

CREATE TABLE IF NOT EXISTS  type_participant(
    id  INTEGER PRIMARY KEY, 
    type VARCHAR(45),
    description TEXT );

CREATE TABLE IF NOT EXISTS person_stratum (
   id_stratum integer REFERENCES stratum (id),
   id_person  integer REFERENCES person(id),
   id_type_participant  integer REFERENCES type_participant(id));

"""
cursor.execute(sql1)
conn.commit()
cursor.execute(sql2)
conn.commit()
cursor.execute(sql3)
conn.commit()
cursor.execute(sql4)
conn.commit()
cursor.execute(sql5)
conn.commit()
cursor.execute(sql6)
conn.commit()
cursor.execute(sql7)
conn.commit()
cursor.execute(sql8)
conn.commit()
cursor.execute(sql9)
conn.commit()

In [ ]:
## Загрузка данных
sql10 = """INSERT INTO person ( id, first_name, last_name, second_name, e_mail, login , password)  VALUES (1, 'Александра', 'Александрова','Александровна', 'Sasha@email.ru', 'Sasha', '123456' );
INSERT INTO person ( id, first_name, last_name, second_name, e_mail, login , password)  VALUES (2, 'Василиса', 'Васильева','Васильевна', 'Vasilisa@email.ru', 'Vasilisa', '234567' );
INSERT INTO person ( id, first_name, last_name, second_name, e_mail, login , password)  VALUES (3, 'Ольга', 'Олегова','Олеговна', 'Olia@email.ru', 'Olia', '345678' );
INSERT INTO person ( id, first_name, last_name, second_name, e_mail, login , password)  VALUES (4, 'Дарья', 'Дариевна','Дарьева', 'Dasha@email.ru', 'Dasha', '456789' );
INSERT INTO person ( id, first_name, last_name, second_name, e_mail, login , password)  VALUES (5, 'Юлия', 'Юльева','Юлиевна', 'Yulia@email.ru', 'Yulia', '567891' );
INSERT INTO person ( id, first_name, last_name, second_name, e_mail, login , password)  VALUES (6, 'Василий', 'Пупкин','Александровна', 'Vasia@email.ru', 'Vasia', '678912' );
INSERT INTO person ( id, first_name, last_name, second_name, e_mail, login , password)  VALUES (7, 'Федор', 'Сумкин','Федорович', 'Fiedor@email.ru', 'Fiedor', '789123' );
INSERT INTO person ( id, first_name, last_name, second_name, e_mail, login , password)  VALUES (8, 'Иван', 'Королев','Васильевич', 'Ivan_K@email.ru', 'Ivan_K', '891234' );
INSERT INTO person ( id, first_name, last_name, second_name, e_mail, login , password)  VALUES (9, 'Иван', 'Царев','Иванович', 'Ivan_C@email.ru', 'Ivan_C', '912345' );
INSERT INTO person ( id, first_name, last_name, second_name, e_mail, login , password)  VALUES (10, 'Джон', 'Доу','Джонович', 'John@email.ru', 'John', 'admin' );

INSERT INTO type_stratum (  id, type, description)  VALUES (1, 'Группа ВУЗ', 'Группа для общего прохождения теста от ВУЗа');

INSERT INTO stratum (  id, name, description, id_creater, ParentId, id_type_stratum)  VALUES (1, 'группа НЯ-1', 'Группа 1 курса факультета "Необычяйные явления"', 10, NULL, 1);
INSERT INTO stratum (  id, name, description, id_creater, ParentId, id_type_stratum)  VALUES (2, 'группа НЛО-1', 'Группа 1 курса факультета "Неопознаных летающих объектов"', 10, NULL, 1);

INSERT INTO type_participant (  id, type, description)  VALUES (1, 'Здающий', 'Персона проходящая тест');
INSERT INTO type_participant (  id, type, description)  VALUES (2, 'Принимающий', 'Персона получающая право на просмотр и редактирование группы и получающая результаты прохождения тестов');

INSERT INTO person_stratum (  id_stratum, id_person, id_type_participant )  VALUES (1 , 1, 1);
INSERT INTO person_stratum (  id_stratum, id_person, id_type_participant )  VALUES (2 , 2, 1);
INSERT INTO person_stratum (  id_stratum, id_person, id_type_participant )  VALUES (1 , 3, 1);
INSERT INTO person_stratum (  id_stratum, id_person, id_type_participant )  VALUES (2 , 4, 1);
INSERT INTO person_stratum (  id_stratum, id_person, id_type_participant )  VALUES (1 , 5, 1);
INSERT INTO person_stratum (  id_stratum, id_person, id_type_participant )  VALUES (2 , 6, 1);
INSERT INTO person_stratum (  id_stratum, id_person, id_type_participant )  VALUES (1 , 7, 1);
INSERT INTO person_stratum (  id_stratum, id_person, id_type_participant )  VALUES (2 , 8, 1);
INSERT INTO person_stratum (  id_stratum, id_person, id_type_participant )  VALUES (1 , 9, 1);
INSERT INTO person_stratum (  id_stratum, id_person, id_type_participant )  VALUES (1 , 10, 2);
INSERT INTO person_stratum (  id_stratum, id_person, id_type_participant )  VALUES (2 , 10, 2);

INSERT INTO test (  id, short_name, description, pass_level_norm, pass_level_good, pass_level_perf, id_creater, ParentId)  
                 VALUES (1, 'Математика', 'Основы математики', 50, 70, 90, 10, NULL);
INSERT INTO test (  id, short_name, description, pass_level_norm, pass_level_good, pass_level_perf, id_creater, ParentId)  
                 VALUES (2, 'Арифметика', 'Основы арифметики', 50, 70, 90, 10, 1);
INSERT INTO test (  id, short_name, description, pass_level_norm, pass_level_good, pass_level_perf, id_creater, ParentId)  
                 VALUES (3, 'Алгебра', 'Основы алгебры', 50, 70, 90, 10, 1);

INSERT INTO type_question (  id, type, description)  VALUES (1, 'Вопрос с еденичным выбором', 'Для правильного ответа нужно выбрать единственный верный вариант ответа');
INSERT INTO type_question (  id, type, description)  VALUES (2, 'Вопрос с множественным выбором', 'Для правильного ответа нужно выбрать все верные и ни одного неверного варианта ответа');
INSERT INTO type_question (  id, type, description)  VALUES (3, 'Вопрос с ответом', 'Для правильного ответа нужно вписать в поле для ответа верный вариант ответа');

INSERT INTO question (  id, text, id_creater, id_type_question)  VALUES (1, 'Чему равно 2+2', 10, 1);
INSERT INTO question (  id, text, id_creater, id_type_question)  VALUES (2, 'Выражение для x при  y=2/x будет равно? ', 10, 1);
INSERT INTO question (  id, text, id_creater, id_type_question)  VALUES (3, 'Чему равно 15/3', 10, 3);
INSERT INTO question (  id, text, id_creater, id_type_question)  VALUES (4, 'Выберете верно сказанное про простые числа ', 10, 2);
INSERT INTO question (  id, text, id_creater, id_type_question)  VALUES (5, 'Есть ли жизнь на Марсе', 10, 1);

INSERT INTO test_question (id_test, id_question) VALUES (1,5);
INSERT INTO test_question (id_test, id_question) VALUES (2,1);
INSERT INTO test_question (id_test, id_question) VALUES (2,3);
INSERT INTO test_question (id_test, id_question) VALUES (3,2);
INSERT INTO test_question (id_test, id_question) VALUES (3,4);

INSERT INTO answer  (id, text, explanation, correct, id_creater, id_question)  VALUES (1, '4', 'Доказательство этого вне пределах знаний автора',100, 10, 1);
INSERT INTO answer  (id, text, explanation, correct, id_creater, id_question)  VALUES (2, '5', 'Доказательство этого вне пределах знаний автора',0, 10, 1);
INSERT INTO answer  (id, text, explanation, correct, id_creater, id_question)  VALUES (3, 'x=2*y', 'Доказательство этого вне пределах знаний автора',0, 10, 2);
INSERT INTO answer  (id, text, explanation, correct, id_creater, id_question)  VALUES (4, 'x=2/y', 'Доказательство этого вне пределах знаний автора',100, 10, 2);
INSERT INTO answer  (id, text, explanation, correct, id_creater, id_question)  VALUES (5, '5', 'Доказательство этого вне пределах знаний автора',100, 10, 3);
INSERT INTO answer  (id, text, explanation, correct, id_creater, id_question)  VALUES (6, 'Они не существуют', 'Доказательство этого вне пределах знаний автора',-1, 10, 4);
INSERT INTO answer  (id, text, explanation, correct, id_creater, id_question)  VALUES (7, 'Они всегда нечетные.', 'Доказательство этого вне пределах знаний автора',1, 10, 4);
INSERT INTO answer  (id, text, explanation, correct, id_creater, id_question)  VALUES (8, 'Деляться только на себя и на 1.', 'Доказательство этого вне пределах знаний автора',1, 10, 4);
INSERT INTO answer  (id, text, explanation, correct, id_creater, id_question)  VALUES (9, 'Есть! Я верю!', 'Доказательство этого вне пределах знаний автора',100, 10, 5);
INSERT INTO answer  (id, text, explanation, correct, id_creater, id_question)  VALUES (10, 'Неизвестно. Ведуться исследования.', 'Доказательство этого вне пределах знаний автора',0, 10, 5);
    
INSERT INTO event_type (  id, type, description)  VALUES (1, 'Незначительное', 'Рядовое событие проходящее переодически');
INSERT INTO event_type (  id, type, description)  VALUES (2, 'Важное', 'Важное событие проходящее достаточно редко и требующее повышенного внимания');

INSERT INTO event (id, description, start_date, finish_date, id_creater, id_type_event)  VALUES (1, 'Контрольная по математике','2018-08-27 12:00:00' ,'2018-08-27 13:00:00', 10, 1);

INSERT INTO event_test(id_event, id_test) VALUES (1,1);    

INSERT INTO event_stratum(id_event, id_stratum, connect_time) VALUES (1,1,'2018-08-27 11:30:00');
INSERT INTO event_stratum(id_event, id_stratum, connect_time) VALUES (1,2,'2018-08-27 11:31:00');

INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 1, 1 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 1, 2 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 1, 3 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 1, 4 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 1, 5 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 2, 6 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 1, 7 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 1, 8 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 1, 9 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 3, 1 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 3, 2 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 3, 3 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 4, 4 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 3, 5 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 3, 6 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 4, 7 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 4, 8 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 4, 9 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES ('5', '2018-08-28 12:35:00', 1, 5, 1 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES ('5', '2018-08-28 12:35:00', 1, 5, 2 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES ('5', '2018-08-28 12:35:00', 1, 5, 3 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES ('5', '2018-08-28 12:35:00', 1, 5, 4 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES ('5', '2018-08-28 12:35:00', 1, 5, 5 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES ('5', '2018-08-28 12:35:00', 1, 5, 6 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES ('5', '2018-08-28 12:35:00', 1, 5, 7 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES ('5', '2018-08-28 12:35:00', 1, 5, 8 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES ('4', '2018-08-28 12:35:00', 1, 5, 9 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 6, 8 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 7, 1 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 7, 2 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 7, 4 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 7, 6 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 7, 7 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 7, 9 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 8, 1 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 8, 2 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 8, 3 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 8, 4 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 8, 5 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 8, 6 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 8, 7 );
INSERT INTO resault_person( value, data, id_event, id_answer, id_person ) VALUES (NULL, '2018-08-28 12:35:00', 1, 8, 9 ); 
"""
cursor.execute(sql10)
conn.commit()

In [44]:
## Запросы

# 1.1 Получить список событий с параметрами
sql_1_1 = """SELECT 
     row_number() OVER () as №, 
     description, 
     start_date, 
     finish_date, 
     person.first_name,
     count(event_test.id_event) as count_test, 
     count(event_stratum.id_event) as count_stratum,
     count(event_person.id_event) as count_free_person   
FROM event 
     LEFT JOIN event_test 
           ON event.id = event_test.id_event  
     LEFT JOIN event_stratum 
          ON event.id = event_stratum.id_event  
     LEFT JOIN event_person
          ON event.id = event_person.id_event
     JOIN person
           ON event.id_creater = person.id 
GROUP BY event.id,   person.first_name;"""
cursor.execute(sql_1_1)
conn.commit()
logger.info(
    "Данные из таблицы:\n{}".format(
        [i for i in cursor.fetchall()])
)

2018-08-26 23:11:14,039 : INFO : Данные из таблицы:
[(1, 'Контрольная по математике', datetime.datetime(2018, 8, 27, 12, 0), datetime.datetime(2018, 8, 27, 13, 0), 'Джон', 2, 2, 0)]


In [45]:
## 1.2 Получить список тестов с параметерами
sql_1_2 = """ 
WITH  count_question_in_test as (
         SELECT 
                test.id as id, 
               count(test_question.id_question) as count 
         FROM test 
                LEFT  JOIN test_question
                     ON test.id = test_question.id_test
        GROUP BY test.id
)
SELECT 
    row_number() OVER () as №, 
    t1.short_name,
    t1.description,
    t1.pass_level_norm  || '%' as pass_norm,
    t1.pass_level_good || '%' as pass_good,
    t1.pass_level_perf || '%' as pass_perf,
    person.first_name || ' ' || person.last_name creator,
    count(t2.id) as count_include_test,
    (SELECT count FROM count_question_in_test WHERE count_question_in_test.id = t1.id ) as count_question_test
    FROM test as t1 
       LEFT JOIN test as t2
            ON t1.id = t2.ParentId 
      LEFT JOIN count_question_in_test as c
            ON t1.id = c.id
      JOIN person
           ON t1.id_creater = person.id 
GROUP BY t1.id,   person.first_name, person.last_name ;"""
cursor.execute(sql_1_2)
conn.commit()
logger.info(
    "Данные из таблицы:\n{}".format(
        [i for i in cursor.fetchall()])
)

2018-08-26 23:11:20,909 : INFO : Данные из таблицы:
[(1, 'Алгебра', 'Основы алгебры', '50%', '70%', '90%', 'Джон Доу', 0, 2), (2, 'Арифметика', 'Основы арифметики', '50%', '70%', '90%', 'Джон Доу', 0, 2), (3, 'Математика', 'Основы математики', '50%', '70%', '90%', 'Джон Доу', 2, 1)]


In [47]:
## 1.3 Получить список участников stratum события
sql_1_3 = """SELECT  s.id, s.name, s.description FROM stratum as s
JOIN event_stratum as es
ON s.id = es.id_stratum
WHERE es.id_event = 1;"""
cursor.execute(sql_1_3)
conn.commit()
logger.info(
    "Данные из таблицы:\n{}".format(
        [i for i in cursor.fetchall()])
)

2018-08-26 23:12:01,512 : INFO : Данные из таблицы:
[(1, 'группа НЯ-1', 'Группа 1 курса факультета "Необычяйные явления"'), (2, 'группа НЛО-1', 'Группа 1 курса факультета "Неопознаных летающих объектов"')]


In [48]:
## 1.4 Получить информацию о персонах 
sql_1_4 = """SELECT * FROM person;"""
cursor.execute(sql_1_4)
conn.commit()
logger.info(
    "Данные из таблицы:\n{}".format(
        [i for i in cursor.fetchall()])
)

2018-08-26 23:12:11,577 : INFO : Данные из таблицы:
[(1, 'Александра', 'Александрова', 'Александровна', 'Sasha@email.ru', 'Sasha', '123456'), (2, 'Василиса', 'Васильева', 'Васильевна', 'Vasilisa@email.ru', 'Vasilisa', '234567'), (3, 'Ольга', 'Олегова', 'Олеговна', 'Olia@email.ru', 'Olia', '345678'), (4, 'Дарья', 'Дариевна', 'Дарьева', 'Dasha@email.ru', 'Dasha', '456789'), (5, 'Юлия', 'Юльева', 'Юлиевна', 'Yulia@email.ru', 'Yulia', '567891'), (6, 'Василий', 'Пупкин', 'Александровна', 'Vasia@email.ru', 'Vasia', '678912'), (7, 'Федор', 'Сумкин', 'Федорович', 'Fiedor@email.ru', 'Fiedor', '789123'), (8, 'Иван', 'Королев', 'Васильевич', 'Ivan_K@email.ru', 'Ivan_K', '891234'), (9, 'Иван', 'Царев', 'Иванович', 'Ivan_C@email.ru', 'Ivan_C', '912345'), (10, 'Джон', 'Доу', 'Джонович', 'John@email.ru', 'John', 'admin')]


In [49]:
## 1.5 Получить список событий для персоны
sql_1_5 = """SELECT * FROM event WHERE id in ( 
  (SELECT e1.id FROM event as e1
     JOIN event_person 
        ON event_person.id_person = 10)
   UNION 
   (SELECT e2.id FROM event as e2
      JOIN event_stratum as es
          ON es.id_event = e2.id
      JOIN person_stratum as ps
          ON es.id_stratum = ps.id_stratum
      WHERE ps.id_person= 10)
);"""
cursor.execute(sql_1_5)
conn.commit()
logger.info(
    "Данные из таблицы:\n{}".format(
        [i for i in cursor.fetchall()])
)

2018-08-26 23:12:14,950 : INFO : Данные из таблицы:
[(1, 'Контрольная по математике', datetime.datetime(2018, 8, 27, 12, 0), datetime.datetime(2018, 8, 27, 13, 0), 10, 1)]


In [50]:
##  1.6 Получить информацию о песонах создающих контент 
sql_1_6 = """SELECT * FROM person 
 JOIN  (
   (SELECT 'event' as type ,event.id_creater as id , count(event.id_creater) FROM event GROUP BY event.id_creater )
  UNION
    (SELECT 'test'as type,  test.id_creater as id  , count(test.id_creater) FROM test GROUP BY test.id_creater)
  UNION
    (SELECT 'stratum'as type, stratum.id_creater as id  , count(stratum.id_creater) FROM stratum GROUP BY stratum.id_creater )
  UNION
    (SELECT 'question'as type, question.id_creater as id  , count(question.id_creater) FROM question GROUP BY question.id_creater)
  UNION
    (SELECT 'answer'as type,answer.id_creater as id  , count(answer.id_creater ) FROM answer GROUP BY answer.id_creater)
) as t ON person.id = t.id;"""
cursor.execute(sql_1_6)
conn.commit()
logger.info(
    "Данные из таблицы:\n{}".format(
        [i for i in cursor.fetchall()])
)

2018-08-26 23:12:18,945 : INFO : Данные из таблицы:
[(10, 'Джон', 'Доу', 'Джонович', 'John@email.ru', 'John', 'admin', 'answer', 10, 10), (10, 'Джон', 'Доу', 'Джонович', 'John@email.ru', 'John', 'admin', 'question', 10, 5), (10, 'Джон', 'Доу', 'Джонович', 'John@email.ru', 'John', 'admin', 'test', 10, 3), (10, 'Джон', 'Доу', 'Джонович', 'John@email.ru', 'John', 'admin', 'event', 10, 1), (10, 'Джон', 'Доу', 'Джонович', 'John@email.ru', 'John', 'admin', 'stratum', 10, 2)]


In [52]:
## 1.7  Получить информацию об объектах созданных персоной
sql_1_7 = """(SELECT 'event' as type ,'' as name, event.description FROM event WHERE event.id_creater = 10)
  UNION
    (SELECT 'test' as type ,'' as name, test.description FROM test WHERE test.id_creater = 10)
  UNION
    (SELECT 'stratum' as type ,'' as name, stratum.description FROM stratum WHERE stratum.id_creater = 10)
  UNION
    (SELECT 'question' as type ,'' as name, question.text as description FROM question WHERE question.id_creater = 10)
  UNION
    (SELECT 'answer' as type ,'' as name, answer.text as description FROM answer WHERE answer.id_creater = 10)
ORDER BY type;"""
cursor.execute(sql_1_7)
conn.commit()
logger.info(
    "Данные из таблицы:\n{}".format(
        [i for i in cursor.fetchall()])
)

2018-08-26 23:12:35,848 : INFO : Данные из таблицы:
[('answer', '', 'Деляться только на себя и на 1.'), ('answer', '', 'Есть! Я верю!'), ('answer', '', 'x=2*y'), ('answer', '', '5'), ('answer', '', 'Неизвестно. Ведуться исследования.'), ('answer', '', '4'), ('answer', '', 'x=2/y'), ('answer', '', 'Они не существуют'), ('answer', '', 'Они всегда нечетные.'), ('event', '', 'Контрольная по математике'), ('question', '', 'Есть ли жизнь на Марсе'), ('question', '', 'Выберете верно сказанное про простые числа '), ('question', '', 'Чему равно 15/3'), ('question', '', 'Чему равно 2+2'), ('question', '', 'Выражение для x при  y=2/x будет равно? '), ('stratum', '', 'Группа 1 курса факультета "Неопознаных летающих объектов"'), ('stratum', '', 'Группа 1 курса факультета "Необычяйные явления"'), ('test', '', 'Основы математики'), ('test', '', 'Основы алгебры'), ('test', '', 'Основы арифметики')]


In [53]:
## 1.8 Получит инфрмацию о группах
sql_1_8 = """SELECT  * FROM stratum;"""
cursor.execute(sql_1_8)
conn.commit()
logger.info(
    "Данные из таблицы:\n{}".format(
        [i for i in cursor.fetchall()])
)

2018-08-26 23:12:41,822 : INFO : Данные из таблицы:
[(1, 'группа НЯ-1', 'Группа 1 курса факультета "Необычяйные явления"', 10, None, 1), (2, 'группа НЛО-1', 'Группа 1 курса факультета "Неопознаных летающих объектов"', 10, None, 1)]


In [54]:
##  1.9 Получить список участников группы
sql_1_9 = """SELECT  * FROM person
       JOIN person_stratum as ps
            ON ps.id_person = person.id
WHERE ps.id_stratum = 1;"""
cursor.execute(sql_1_9)
conn.commit()
logger.info(
    "Данные из таблицы:\n{}".format(
        [i for i in cursor.fetchall()])
)

2018-08-26 23:12:45,684 : INFO : Данные из таблицы:
[(1, 'Александра', 'Александрова', 'Александровна', 'Sasha@email.ru', 'Sasha', '123456', 1, 1, 1), (3, 'Ольга', 'Олегова', 'Олеговна', 'Olia@email.ru', 'Olia', '345678', 1, 3, 1), (5, 'Юлия', 'Юльева', 'Юлиевна', 'Yulia@email.ru', 'Yulia', '567891', 1, 5, 1), (7, 'Федор', 'Сумкин', 'Федорович', 'Fiedor@email.ru', 'Fiedor', '789123', 1, 7, 1), (9, 'Иван', 'Царев', 'Иванович', 'Ivan_C@email.ru', 'Ivan_C', '912345', 1, 9, 1), (10, 'Джон', 'Доу', 'Джонович', 'John@email.ru', 'John', 'admin', 1, 10, 2)]


In [55]:
## 1.10 Получить список событий для группы
sql_1_10 = """SELECT * FROM event as e 
    JOIN event_stratum as es
         ON e.id = es.id_event
WHERE es.id_stratum = 2;"""
cursor.execute(sql_1_10)
conn.commit()
logger.info(
    "Данные из таблицы:\n{}".format(
        [i for i in cursor.fetchall()])
)

2018-08-26 23:12:50,285 : INFO : Данные из таблицы:
[(1, 'Контрольная по математике', datetime.datetime(2018, 8, 27, 12, 0), datetime.datetime(2018, 8, 27, 13, 0), 10, 1, 1, 2, datetime.datetime(2018, 8, 27, 11, 31))]


In [56]:
##  1.11 Получить список тестов в событии 
sql_1_11 = """SELECT * FROM test as t
    JOIN event_test as et
        ON et.id_test = t.id
WHERE et.id_event = 1;"""
cursor.execute(sql_1_11)
conn.commit()
logger.info(
    "Данные из таблицы:\n{}".format(
        [i for i in cursor.fetchall()])
)

2018-08-26 23:12:54,231 : INFO : Данные из таблицы:
[(1, 'Математика', 'Основы математики', 50, 70, 90, 10, None, 1, 1)]


In [57]:
##  2.2 вьюшки
##  2.1 Получить status прохождения евентов 
sql_2_1 = """
CREATE OR REPLACE VIEW status_event as 
       SELECT event.id, event.description,
            CASE WHEN event.finish_date < NOW() THEN  'завершено'
                         WHEN event.start_date > NOW() THEN 'ожидается начало'
                         ELSE 'в процессе'
            END
        FROM event;
SELECT * FROM status_event;"""
cursor.execute(sql_2_1)
conn.commit()
logger.info(
    "Данные из таблицы:\n{}".format(
        [i for i in cursor.fetchall()])
)

2018-08-26 23:12:58,848 : INFO : Данные из таблицы:
[(1, 'Контрольная по математике', 'ожидается начало')]


In [59]:
## 2.2 Получить список персон и статус прохождения ими тестов
sql_2_2 = """
CREATE OR REPLACE VIEW status_person_event as 
SELECT 
        p.id, 
        p.first_name , 
       CASE 
              WHEN count(rp.id_answer) > 0 THEN 'приступил'
              ELSE 'ожидается первый ответ' 
       END
FROM person as p
     LEFT JOIN  resault_person as rp 
          ON  p.id = rp.id_person
     JOIN ((SELECT e.id as id_event, ep.id_person
           FROM event as e
              JOIN event_person as ep
                    ON e.id = ep.id_event)
          UNION
          (SELECT e.id as id_event, ps.id_person 
           FROM event as e
            JOIN event_stratum as es
                  ON   e.id = es.id_event
             JOIN person_stratum as ps
                   ON es.id_stratum=ps.id_stratum)
          ) as t
    ON t.id_person = p.id
    GROUP BY p.id;

SELECT * FROM status_person_event;"""
cursor.execute(sql_2_2)
conn.commit()
logger.info(
    "Данные из таблицы:\n{}".format(
        [i for i in cursor.fetchall()])
)

2018-08-26 23:13:24,610 : INFO : Данные из таблицы:
[(4, 'Дарья', 'приступил'), (10, 'Джон', 'ожидается первый ответ'), (9, 'Иван', 'приступил'), (7, 'Федор', 'приступил'), (6, 'Василий', 'приступил'), (3, 'Ольга', 'приступил'), (1, 'Александра', 'приступил'), (5, 'Юлия', 'приступил'), (2, 'Василиса', 'приступил'), (8, 'Иван', 'приступил')]


In [ ]:
## Работа pandas

In [83]:
## получаем данные из базы данных

sql_event = "SELECT * FROM event;"
cursor.execute(sql_event)
conn.commit()
event = [i for i in cursor.fetchall()]

sql_test = "SELECT * FROM test;"
cursor.execute(sql_test)
conn.commit()
test = [i for i in cursor.fetchall()]

sql_person = "SELECT * FROM person;"
cursor.execute(sql_person)
conn.commit()
person = [i for i in cursor.fetchall()]

sql_question = "SELECT * FROM question;"
cursor.execute(sql_question)
conn.commit()
question = [i for i in cursor.fetchall()]

sql_answer = "SELECT * FROM answer;"
cursor.execute(sql_answer)
conn.commit()
answer = [i for i in cursor.fetchall()]

sql_stratum = "SELECT * FROM stratum;"
cursor.execute(sql_stratum)
conn.commit()
stratum = [i for i in cursor.fetchall()]

sql_resault_person  = "SELECT * FROM resault_person ;"
cursor.execute(sql_resault_person)
conn.commit()
resault_person  = [i for i in cursor.fetchall()]

sql_event_test  = "SELECT * FROM event_test ;"
cursor.execute(sql_event_test)
conn.commit()
event_test = [i for i in cursor.fetchall()]

sql_event_person  = "SELECT * FROM event_person ;"
cursor.execute(sql_event_person)
conn.commit()
event_person = [i for i in cursor.fetchall()]

sql_event_stratum  = "SELECT * FROM event_stratum ;"
cursor.execute(sql_event_stratum)
conn.commit()
event_stratum = [i for i in cursor.fetchall()]

sql_test_question  = "SELECT * FROM test_question ;"
cursor.execute(sql_test_question)
conn.commit()
test_question = [i for i in cursor.fetchall()]


In [86]:
## переводим в датафрейиы
df_event = pd.DataFrame.from_records(event, columns=['id','description', 'start_date', 'finish_date', 'id_creator', 'id_type_event'])
df_test = pd.DataFrame.from_records(test, columns=['id','short_name','description', 'pass_level_norm', 'pass_level_good', 'pass_level_pref', 'id_creator', 'ParentId'])
df_person = pd.DataFrame.from_records(person, columns=['id','first_name', 'last_name', 'second_name', 'e_mail', 'login','password'])
df_question = pd.DataFrame.from_records(question, columns=['id','text', 'id_creator', 'type'])
df_answer = pd.DataFrame.from_records(answer, columns=['id','text', 'explanation', 'correct', 'id_creator', 'id_question'])
df_stratum = pd.DataFrame.from_records(stratum, columns=['id','name', 'description',  'id_creator','ParentId', ' id_type_stratum'])
df_resault_person = pd.DataFrame.from_records(resault_person, columns=['value','data', 'id_event', 'id_answer', 'id_person'])
df_event_test = pd.DataFrame.from_records(event_test, columns=['id_event','id_test'])
df_event_person = pd.DataFrame.from_records(event_person, columns=['id_event','id_person','connect_date'])
df_event_stratum = pd.DataFrame.from_records(event_stratum, columns=['id_event','id_stratum','connect_date'])
df_test_question = pd.DataFrame.from_records(test_question, columns=['id_test','id_question'])


In [87]:
## реализация запроса 1.1

